# Face Recognition - Advance Approach

## Install Dependencies and Setup

In [ ]:
!pip install ultralytics

## Importing required packages

In [ ]:
import os
import json
from ultralytics import YOLO

## Accessories

In [4]:
#  images and JSON labels
data_folder = '/content/FaceRecognition2/data/'

# to convert JSON labels to YOLO format
def convert_to_yolo_format(json_data, img_width, img_height):
    yolo_labels = []
    for shape in json_data['shapes']:
        if shape['label'] == 'face':
            points = shape['points']

            x1, y1 = points[0]
            x2, y2 = points[1]


            x_center = (x1 + x2) / 2 / img_width
            y_center = (y1 + y2) / 2 / img_height
            width = (x2 - x1) / img_width
            height = (y2 - y1) / img_height


            yolo_labels.append(f"0 {x_center} {y_center} {width} {height}")
    return yolo_labels


for filename in os.listdir(data_folder):
    if filename.endswith('.json'):
        json_path = os.path.join(data_folder, filename)
        with open(json_path, 'r') as f:
            data = json.load(f)

        img_width = data['imageWidth']
        img_height = data['imageHeight']
        yolo_labels = convert_to_yolo_format(data, img_width, img_height)

        # YOLO labels to a .txt file
        txt_filename = filename.replace('.json', '.txt')
        txt_path = os.path.join(data_folder, txt_filename)
        with open(txt_path, 'w') as f:
            f.write('\n'.join(yolo_labels))


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
def convert_to_yolo_format(json_file, img_width, img_height):
    """Converts JSON bounding boxes to YOLO format."""
    yolo_data = []
    with open(json_file, 'r') as f:
        data = json.load(f)
        shapes = data['shapes']

        for shape in shapes:
            label = 0
            points = shape['points']
            x_min, y_min = points[0]
            x_max, y_max = points[1]

            # Compute YOLO format
            x_center = (x_min + x_max) / 2.0 / img_width
            y_center = (y_min + y_max) / 2.0 / img_height
            width = (x_max - x_min) / img_width
            height = (y_max - y_min) / img_height

            yolo_data.append(f"{label} {x_center} {y_center} {width} {height}")

    return yolo_data

def convert_json_to_txt(label_dir, output_dir, img_dir):
    """Converts all JSON label files in a directory to YOLO .txt format."""
    for json_filename in os.listdir(label_dir):
        if json_filename.endswith('.json'):
            json_path = os.path.join(label_dir, json_filename)
            with open(json_path, 'r') as f:
                data = json.load(f)
                image_path = os.path.join(img_dir, os.path.basename(data['imagePath']))
                img_width = data['imageWidth']
                img_height = data['imageHeight']

                yolo_data = convert_to_yolo_format(json_path, img_width, img_height)


                output_txt_path = os.path.join(output_dir, json_filename.replace('.json', '.txt'))
                with open(output_txt_path, 'w') as txt_file:
                    txt_file.write("\n".join(yolo_data))

# Set your paths
label_dir_train = '/content/FaceRecognition2/data/train/labels'
label_dir_val = '/content/FaceRecognition2/data/val/labels'
img_dir_train = '/content/FaceRecognition2/data/train/images'
img_dir_val = '/content/FaceRecognition2/data/val/images'
output_dir_train = label_dir_train
output_dir_val = label_dir_val

# Convert train and val sets
convert_json_to_txt(label_dir_train, output_dir_train, img_dir_train)
convert_json_to_txt(label_dir_val, output_dir_val, img_dir_val)


## Model Training

In [ ]:
# Load a YOLOv8n model, available variants are yolov8n, yolov8s, yolov8m, yolov8l, yolov8x
model = YOLO('yolov8n.pt')


model.train(data='/content/FaceRecognition2/data/config.yaml', epochs=50, imgsz=640, batch=32)


Ultralytics 8.3.23 🚀 Python-3.10.12 torch-2.5.0+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/FaceRecognition2/data/config.yaml, epochs=100, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=T

train: Scanning /content/FaceRecognition2/data/train/labels.cache... 52 images, 8 backgrounds, 0 corrupt: 100%|██████████| 60/60 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.20 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
val: Scanning /content/FaceRecognition2/data/val/labels.cache... 15 images, 0 backgrounds, 0 corrupt: 100%|██████████| 15/15 [00:00<?, ?it/s]


Plotting labels to runs/detect/train3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/train3
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     86/100         0G     0.7337     0.5691      1.013         54        640: 100%|██████████| 2/2 [00:58<00:00, 29.32s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:03<00:00,  3.25s/it]

                   all         15         15      0.997          1      0.995      0.704



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     87/100         0G     0.6114     0.4875     0.9316         63        640:  50%|█████     | 1/2 [00:31<00:31, 31.53s/it]

## Model Evaluation

In [ ]:
# Validate the model
metrics = model.val()
print(metrics)


Ultralytics 8.3.19 🚀 Python-3.11.10 torch-2.2.2+cu121 CPU (Intel Core(TM) i5-8250U 1.60GHz)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /home/waqar/scrapper/Aziz_facedector/FaceRecognition2/data/data/labels/val.cache... 15 images, 0 backgrounds, 0 corrupt: 100%|██████████| 15/15 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:02<00:00,  1.04s/it]


                   all         15         15          1      0.996      0.995      0.778
Speed: 3.4ms preprocess, 123.0ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to runs/detect/train22
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x784b90b56510>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.

## Model Testing

In [ ]:
import cv2
import json
import torch

model = torch.load('/content/runs/detect/train/weights/best.pt')  # Replace with your model loading code

# Initialize webcam
cap = cv2.VideoCapture(0)

# Real-time detection loop
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Perform detection on the frame
    results = model(frame)

    # Extract predictions and format for display and output
    predictions = []
    group_id_counter = 1

    for pred in results[0].boxes:
        bbox = pred.xyxy[0].cpu().numpy().tolist()

        group_id = getattr(pred, 'group_id', None)
        if group_id is None:
            group_id = group_id_counter
            group_id_counter += 1

        # Append to predictions
        predictions.append({
            "label": "face",
            "points": [
                [bbox[0], bbox[1]],
                [bbox[2], bbox[3]]
            ],
            "group_id": group_id,
            "description": "",
            "shape_type": "rectangle",
            "flags": {}
        })

        # Draw bounding boxes on the frame
        cv2.rectangle(
            frame,
            (int(bbox[0]), int(bbox[1])),
            (int(bbox[2]), int(bbox[3])),
            (255, 0, 0), 2
        )
        cv2.putText(frame, "face", (int(bbox[0]), int(bbox[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    # Print real-time JSON predictions
    json_output = json.dumps(predictions, indent=4)
    print(json_output)

    # Display the live video feed with detections
    cv2.imshow('Real-Time Detection', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

## Conclusion

YOLO is performing very well with the comparison to other models, so the best model for face recognition is YOLO.